In [1]:
from sklearn import datasets


iris = datasets.load_iris()
X = iris.data
y = iris.target

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

In [3]:
import sys
sys.path.append("../") 

# Stochastic Gradient Descent

In [4]:
import mxnet as mx
import numpy as np
from mxnet import ndarray as nd
from hamiltonian.inference.sgd import sgd
from hamiltonian.models.softmax import softmax

model_ctx=mx.cpu()
hyper={'alpha':1}
in_units=X.shape[1]
out_units=len(np.unique(y))
model=softmax(hyper,in_units,out_units,ctx=model_ctx)
inference=sgd(model,model.par,step_size=0.01,ctx=model_ctx)

In [5]:
par,loss=inference.fit(epochs=1000,batch_size=60,gamma=0.9,
                       X_train=nd.array(X_train,ctx=model_ctx),y_train=nd.array(y_train,ctx=model_ctx),verbose=False)

100%|██████████| 1000/1000 [00:02<00:00, 453.61it/s]


In [6]:
import hamiltonian
import importlib

try:
    importlib.reload(hamiltonian.models.softmax)
    importlib.reload(hamiltonian.inference.sgd)
    print('modules re-loaded')
except:
    print('no modules loaded yet')

modules re-loaded


In [7]:
model.net.collect_params()

{'0.weight': Parameter (shape=(3, 4), dtype=float32),
 '0.bias': Parameter (shape=(3,), dtype=float32)}

In [8]:
y_pred=model.predict(model.par,nd.array(X_test,ctx=model_ctx))

In [9]:
from sklearn.metrics import classification_report

print(classification_report(np.int32(y_test),np.int32(y_pred.sample().asnumpy())))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      0.82      0.90        11
           2       0.85      1.00      0.92        11

    accuracy                           0.93        30
   macro avg       0.95      0.94      0.94        30
weighted avg       0.94      0.93      0.93        30



# Hamiltonian Monte Carlo

In [10]:
model_ctx=mx.cpu()
hyper={'alpha':25.}
in_units=X.shape[1]
out_units=len(np.unique(y))

In [11]:
from hamiltonian.inference.hmc import hmc
from hamiltonian.models.softmax import softmax

model=softmax(hyper,in_units,out_units,ctx=model_ctx)
inference=hmc(model,model.par,step_size=0.1,ctx=model_ctx)

In [12]:
import hamiltonian
import importlib

try:
    importlib.reload(hamiltonian.models.softmax)
    importlib.reload(hamiltonian.inference.hmc)
    print('modules re-loaded')
except:
    print('no modules loaded yet')

modules re-loaded


In [13]:
samples=inference.sample(epochs=3000,burn_in=1000,path_length=1.0,chains=4,X_train=nd.array(X_train,ctx=model_ctx),y_train=nd.array(y_train,ctx=model_ctx))

100%|██████████| 4000/4000 [00:35<00:00, 112.02it/s]


In [14]:
len(samples)

4

In [15]:
import arviz as az

datasets=[az.convert_to_inference_data(sample) for sample in samples]

In [16]:
dataset = az.concat(datasets, dim="chain")

In [17]:
az.summary(dataset)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
"0.weight[0,0]",-44.956,14.383,-74.255,-29.546,6.999,5.362,4.0,7.0,3.52
"0.weight[0,1]",-12.059,13.425,-36.097,8.239,6.332,4.805,5.0,10.0,3.02
"0.weight[0,2]",-5.426,5.261,-16.113,5.033,1.881,1.383,7.0,11.0,1.73
"0.weight[0,3]",-6.303,16.322,-30.813,17.941,8.075,6.175,4.0,5.0,3.88
"0.weight[1,0]",-32.276,21.856,-61.476,0.096,10.614,8.092,5.0,6.0,2.71
"0.weight[1,1]",-9.308,16.994,-30.901,11.179,8.317,6.349,5.0,4.0,3.15
"0.weight[1,2]",19.565,8.107,7.566,29.625,3.883,2.954,5.0,11.0,2.65
"0.weight[1,3]",-29.402,7.464,-39.939,-12.967,3.398,2.588,5.0,4.0,2.43
"0.weight[2,0]",14.097,5.644,3.203,23.943,2.221,1.647,6.0,12.0,1.66
"0.weight[2,1]",-11.044,20.747,-37.249,26.693,10.256,7.843,5.0,8.0,3.38


# PyMC3

In [19]:
import pymc3 as pm
import theano.tensor as tt

In [22]:
with pm.Model() as model:
    alpha = pm.Normal('alpha', mu=0, sd=5, shape=3)
    beta = pm.Normal('beta', mu=0, sd=5, shape=(4,3))
    mu = pm.Deterministic('mu', alpha + pm.math.dot(X_train, beta))
    theta = tt.nnet.softmax(mu)
    y_hat = pm.Categorical('y_hat', p=theta, observed=y_train)
    trace_s = pm.sample(2000)

WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta, alpha]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 81 seconds.
There were 266 divergences after tuning. Increase `target_accept` or reparameterize.
There were 272 divergences after tuning. Increase `target_accept` or reparameterize.
There were 317 divergences after tuning. Increase `target_accept` or reparameterize.
There were 427 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 10% for some parameters.


In [24]:
az.summary(trace_s,var_names=["alpha", "beta"])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha[0],1.109,4.913,-7.762,10.432,0.226,0.160,498.0,751.0,1.01
alpha[1],4.529,4.465,-2.828,13.372,0.328,0.232,208.0,585.0,1.02
alpha[2],-5.535,4.041,-13.071,2.113,0.110,0.078,1300.0,1394.0,1.02
"beta[0,0]",1.918,3.540,-4.650,8.760,0.140,0.099,644.0,1237.0,1.02
"beta[0,1]",0.876,3.121,-4.801,6.874,0.114,0.081,755.0,1176.0,1.01
"beta[0,2]",-2.448,3.198,-8.691,3.249,0.141,0.100,513.0,1551.0,1.01
"beta[1,0]",4.830,4.026,-1.753,12.752,0.225,0.159,316.0,1765.0,1.01
"beta[1,1]",-0.782,3.304,-7.066,5.150,0.174,0.123,381.0,1067.0,1.02
"beta[1,2]",-4.240,3.320,-10.575,2.185,0.112,0.079,885.0,1181.0,1.01
"beta[2,0]",-6.674,3.582,-13.903,-0.192,0.119,0.090,895.0,987.0,1.01
